Importation des library

In [2]:
import numpy as np
import pandas as pd
import vectorbt as vbt
from itertools import product
# ============================================================
# CONFIGURATION DATASETS
# ============================================================
pd.set_option('future.no_silent_downcasting', True)


DATASETS = {
    "BTCUSDT_4h": r"C:\Users\gunsa\Desktop\Git-Repo\Bot-Trading\Data\4h\BTCUSDT\BTCUSDT_1years_2024-12-22_TO_2025-12-25_4h_data",
    "ETHUSDT_4h": r"C:\Users\gunsa\Desktop\Git-Repo\Bot-Trading\Data\4h\ETHUSDT\ETHUSDT_1years_2024-12-22_TO_2025-12-26_4h_data",
    "BTCUSDT_1h": r"C:\Users\gunsa\Desktop\Git-Repo\Bot-Trading\Data\1h\BTCUSDT\BTCUSDT_1years_2024-12-22_TO_2025-12-26_1h_data",
    "ETHUSDT_1h": r"C:\Users\gunsa\Desktop\Git-Repo\Bot-Trading\Data\1h\ETHUSDT\ETHUSDT_3years_2022-12-22_TO_2025-12-26_1h_data",
}

# ============================================================
# PARAMÈTRES GÉNÉRAUX
# ============================================================

INIT_CASH = 1000
FEES = 0.001

MIN_DIV_BARS = 10
MAX_DIV_BARS = 200

# ============================================================
# GRID SEARCH
# ============================================================

PARAM_GRID = {
    "rsi_length": [10, 14, 21],
    "lookback": [(2, 2), (3, 3), (4, 4)],
    "stop_loss": [0.02, 0.03, 0.04],
    "take_profit": [0.04, 0.06, 0.08],
}

# ============================================================
# STRATÉGIE RSI DIVERGENCES (LONG ONLY)
# ============================================================

def run_rsi_divergence_strategy(
    df,
    rsi_length,
    lookback_left,
    lookback_right,
    stop_loss_pct,
    take_profit_pct,
):
    high = df["High"].astype(np.float64)
    low = df["Low"].astype(np.float64)
    close = df["Close"].astype(np.float64).ffill().infer_objects(copy=False)

    rsi = vbt.RSI.run(close, window=rsi_length).rsi

    window = lookback_left + lookback_right + 1

    pivot_low_price = (
        low.shift(lookback_right) == low.rolling(window).min()
    )
    pivot_low_rsi = (
        rsi.shift(lookback_right) == rsi.rolling(window).min()
    )

    bullish_div = pd.Series(False, index=df.index)
    hidden_bullish_div = pd.Series(False, index=df.index)

    idx = np.where(pivot_low_price & pivot_low_rsi)[0]

    if len(idx) >= 2:
        idx1 = idx[1:]
        idx2 = idx[:-1]
        bars_between = np.abs(idx1 - idx2)

        valid_regular = (
            (low.iloc[idx1].values < low.iloc[idx2].values) &
            (rsi.iloc[idx1].values > rsi.iloc[idx2].values) &
            (bars_between >= MIN_DIV_BARS) &
            (bars_between <= MAX_DIV_BARS)
        )

        valid_hidden = (
            (low.iloc[idx1].values > low.iloc[idx2].values) &
            (rsi.iloc[idx1].values < rsi.iloc[idx2].values) &
            (bars_between >= MIN_DIV_BARS) &
            (bars_between <= MAX_DIV_BARS)
        )

        sig_reg = idx1[valid_regular] - lookback_right
        sig_hid = idx1[valid_hidden] - lookback_right

        bullish_div.iloc[sig_reg[sig_reg >= 0]] = True
        hidden_bullish_div.iloc[sig_hid[sig_hid >= 0]] = True

    entries = (bullish_div | hidden_bullish_div)
    entries = entries.shift(1).fillna(False).astype(bool)

    pf = vbt.Portfolio.from_signals(
        close=close,
        entries=entries,
        sl_stop=stop_loss_pct,
        tp_stop=take_profit_pct,
        init_cash=INIT_CASH,
        fees=FEES,
        direction="longonly"
    )

    return pf

# ============================================================
# GRID SEARCH MULTI-DATASETS (AVEC DÉTAIL)
# ============================================================

def grid_search_multi_dataset(datasets):
    results = []

    grid = product(
        PARAM_GRID["rsi_length"],
        PARAM_GRID["lookback"],
        PARAM_GRID["stop_loss"],
        PARAM_GRID["take_profit"],
    )

    for rsi_len, (lb_l, lb_r), sl, tp in grid:
        row = {
            "rsi_length": rsi_len,
            "lookback_left": lb_l,
            "lookback_right": lb_r,
            "stop_loss": sl,
            "take_profit": tp,
        }

        returns = []
        drawdowns = []
        trades = []

        for name, path in datasets.items():
            df = pd.read_parquet(path)

            pf = run_rsi_divergence_strategy(
                df,
                rsi_len,
                lb_l,
                lb_r,
                sl,
                tp
            )

            stats = pf.stats()

            row[f"return_{name}"] = stats["Total Return [%]"]
            row[f"drawdown_{name}"] = stats["Max Drawdown [%]"]
            row[f"trades_{name}"] = pf.trades.count()

            returns.append(stats["Total Return [%]"])
            drawdowns.append(stats["Max Drawdown [%]"])
            trades.append(pf.trades.count())

        row["mean_return"] = np.mean(returns)
        row["mean_drawdown"] = np.mean(drawdowns)
        row["mean_trades"] = np.mean(trades)

        row["score"] = row["mean_return"] / (abs(row["mean_drawdown"]) + 1e-6)

        results.append(row)

    return (
        pd.DataFrame(results)
        .sort_values("score", ascending=False)
        .reset_index(drop=True)
    )

# ============================================================
# MAIN
# ============================================================

if __name__ == "__main__":
    print("▶ Lancement Grid Search multi-datasets...\n")

    results = grid_search_multi_dataset(DATASETS)

    print("===== TOP 10 PARAMÈTRES ROBUSTES (DÉTAILLÉS) =====\n")
    print(results.head(10))

    results.to_parquet("grid_search_results_detailed.parquet")

▶ Lancement Grid Search multi-datasets...



c:\Users\gunsa\AppData\Local\Programs\Python\Python312\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\gunsa\AppData\Local\Programs\Python\Python312\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\Users\gunsa\AppData\Local\Programs\Python\Python312\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

c:\Users\gunsa\AppData\Local\Programs\Python\Python312\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

c:\Users\gunsa\AppData\Local\Programs\Python\Python312\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\gunsa\AppData\Local\Programs\Python\Python312\Lib\site-packages\vectorbt\generic\stats_builder.py:396

===== TOP 10 PARAMÈTRES ROBUSTES (DÉTAILLÉS) =====

   rsi_length  lookback_left  lookback_right  stop_loss  take_profit  \
0          10              3               3       0.03         0.06   
1          10              3               3       0.02         0.06   
2          10              3               3       0.03         0.04   
3          10              3               3       0.04         0.06   
4          10              3               3       0.02         0.08   
5          10              3               3       0.04         0.04   
6          14              3               3       0.02         0.06   
7          10              4               4       0.02         0.06   
8          10              4               4       0.03         0.06   
9          10              3               3       0.03         0.08   

   return_BTCUSDT_4h  drawdown_BTCUSDT_4h  trades_BTCUSDT_4h  \
0          -6.391095            22.280390                 19   
1           9.284668       